# F1 Lap-by-Lap Race Prediction 5: 2 + 3
Data from http://ergast.com/mrd/db/  

This model uses an LSTM to predict the positions, laptimes and pitstops 20 drivers. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
db_dir = '/content/drive/My Drive/f1ml/db/'

In [ ]:
# The time format in the Ergast database is MM:SS.ms
def time_to_int(t):
  if (t == float):
    return t
  t2 = str(t)
  ts = t2.rsplit(':')
  if ('\\N' in t2):
    return None
  if (not '.' in t2):
    return None
  if (len(ts) > 1):
    return int(ts[0]) * 60 + float(ts[1])
  else:
    return float(ts[0])

### Helper functions for getting information from ids

In [ ]:
races = pd.read_csv(db_dir + 'races.csv')
circuits = pd.read_csv(db_dir + 'circuits.csv')
drivers = pd.read_csv(db_dir + 'drivers.csv')
constructor = pd.read_csv(db_dir + 'constructors.csv')
status = pd.read_csv(db_dir + 'status.csv')

def race_info(raceId):
  _races = races
  _r = _races.query(f'raceId  == {raceId}')
  if (_r.empty):
    return None, None, None
  _year = _r['year'].item()
  _round = _r['round'].item()
  _circuitId = _r['circuitId'].item()
  return _year, _round, _circuitId

def circuit_info(circuitId):
  _circuits = circuits
  _c = _circuits.query(f'circuitId  == {circuitId}')
  if (_c.empty):
    return None, None, None
  _name = _c['name'].item()
  _location = _c['location'].item()
  _country = _c['country'].item()
  return _name, _location, _country

def driver_info(id):
  _drivers = drivers
  _d = _drivers.query(f'driverId  == {id}')
  if (_d.empty):
    return None, None, None, None, None, None
  _number = _d['number'].item()
  _code = _d['code'].item()
  _forename = _d['forename'].item()
  _surname = _d['surname'].item()
  _dob = _d['dob'].item()
  _nationality = _d['nationality'].item()
  return _number, _code, _forename, _surname, _dob, _nationality

def constructor_info(id):
  _constructor = constructors
  _c = _constructor.query(f'constructorId  == {id}')
  if (_c.empty):
    return None, None
  _name = _d['name'].item()
  _nationality = _d['nationality'].item()
  return _name, _nationality

def status_info(id):
  _status = status
  _s = _status.query(f'statusId == {id}')
  if (_s.empty):
    return None
  _sstr = _s['status'].item()
  return _sstr

## Data preparation

Each csv file contains information of one race, and each row contains information of a lap. 

We are using the files prepared by the first notebook.  

In [ ]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(db_dir + f'races_npy/{year}'):
        os.makedirs(db_dir + f'races_npy/{year}')

### Status Embedding

In [ ]:
stat_emb = [
  [4.0, 3.0, 130.0], # Accident/Collision
  [22.0, 5.0, 10.0, 23.0, 44.0, 47.0, 30.0, 32.0, 8.0, 38.0, 43.0, 85.0, 9.0, 86.0, 6.0, 2.0, 7.0, 87.0, 71.0, 41.0, 46.0, 37.0, 65.0, 78.0, 25.0, 74.0, 75.0, 26.0, 51.0, 40.0, 79.0, 36.0, 83.0, 80.0, 21.0, 69.0, 72.0, 70.0, 27.0, 60.0, 63.0, 29.0, 64.0, 66.0, 56.0, 59.0, 61.0, 42.0, 39.0, 48.0, 49.0, 34.0, 35.0, 28.0, 24.0, 33.0, 129.0, 76.0, 91.0, 131.0, 101.0, 132.0, 135.0,  84.0,  136.0,  105.0,  137.0,  138.0,  139.0], # Car issues
  [11.0,  13.0,  12.0,  14.0,  17.0,  15.0,  16.0, 18.0,  55.0,  58.0,  45.0, 88.0], # Lapped
  [0.0], # No problem
  [77.0, 73.0, 82.0, 81.0, 62.0, 54.0, 31.0, 96.0], # Other
  [20.0] #'Spun off'
] 

def stat_embed(id):
  _emb = np.zeros(6)
  for i in range(6):
    if id in stat_emb[i]:
      _emb[i] = 1
      return _emb
  _emb[4] = 1
  return _emb # Other

def stat_unbed(array, retired=False):
  _a = np.copy(array)
  if (retired):
    _a[3] = 0
  _i = np.argmax(_a)
  if (_i == 0):
    return 'Accident/Collision'
  elif (_i == 1):
    return 'Car Issues'
  elif (_i == 2):
    return 'Lapped'
  elif (_i == 3):
    return 'No Problem'
  elif (_i == 4):
    return 'Other'
  elif (_i == 5):
    return 'Spun off'
  else:
    return 'something is wrong'

### laptime embedding

In [ ]:
def lt_embed(laptime):
  # laptime should be a float with 3 decimal places
  _lt = math.floor(laptime * 10)
  _lt_emb = []
  _ret = []
  for i in range(4):
    _lt_emb.append(int(_lt % 10))
    _lt = math.floor(_lt / 10)
  _ret = np.zeros(2)
  if (_lt_emb[-1] == 1):
    _ret[0] = 1
  elif (_lt_emb[-1] == 2):
    _ret[1] = 1
  elif (_lt_emb[-1] > 2):
    _ret[0] = 1
    _ret[1] = 1
  for i in range(3):
    _t = np.zeros(10)
    _t[_lt_emb[2 - i]] = 1
    _ret = np.append(_ret, _t)
  return _ret

def lt_unbed(l_array):
  _ret = 0
  if (l_array[0] >= 0.5 and l_array[1] >= 0.5):
    _ret += 300
  elif (l_array[0] >= 0.5):
    _ret += 100
  elif (l_array[1] >= 0.5):
    _ret += 200
  _ret += np.argmax(l_array[2:12]) * 10
  _ret += np.argmax(l_array[12:22]) * 1
  _ret += np.argmax(l_array[22:32]) * 0.1
  return _ret

In [ ]:
l = lt_embed(123.456)
print(len(l))

32


In [ ]:
u = lt_unbed(l)
print(u)

123.4


### driverId embedding

In [ ]:
drivers_short = pd.read_csv(db_dir + 'drivers_short.csv')
# from driverId to our id
def driver_embed_idx(driverId):
  row = drivers_short.query(f'driverId == {driverId}').index
  if (row.empty):
    return 0
  return row.item() + 1

# from our id to driverId
def driver_unbed_idx(idx):
  row = drivers_short.iloc[idx-1]
  return row['driverId']

# from our id to array
def driver_embed(idx):
  _e = np.zeros(130)
  _e[idx-1] = 1
  return _e

# from array to our id
def driver_unbed(d_array):
  return np.argmax(d_array) + 1

### embed to npy

In [ ]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(db_dir + f'races_npy/{year}'):
      os.makedirs(db_dir + f'races_npy/{year}')

  cur_year = os.listdir(db_dir + f'races/{year}/')
  counter = 0
  for round in cur_year:
    
    cur_race = pd.read_csv(db_dir + f'races/{year}/{round}')

    #if not os.path.exists(db_dir + f'races_npy/{year}/{round}_in.npy'):
    if (True):
      for i in range(len(cur_race)-1):
        # Size([141])
        cur = cur_race.iloc[i].values[1:142] # removes index and trim size
        # replace NaN with 0
        cur[cur != cur] = 0
        cur_ret = np.zeros(130)
        cur_ret[int(cur[0])] = 1 # circuitId
        lap_percentage = float(i) / (len(cur_race)-1)
        _lp = np.zeros(1)
        _lp[0] = lap_percentage
        cur_ret = np.append(cur_ret, _lp)
        for k in range(1, 141, 7):
    
          d_id = driver_embed(driver_embed_idx(int(cur[k])))
          cur_ret = np.append(cur_ret, d_id)
    
          # 2 bits: top or top3 or top10
          d_s = np.zeros(3)
          _s = int(cur[k+1])-1
          if (_s <= 1):
            d_s[0] = 1
          if (_s <=3):
            d_s[1] = 1
          if (_s <=10):
            d_s[2] = 1
          cur_ret = np.append(cur_ret, d_s)
    
          c_s = np.zeros(2)
          _c = int(cur[k+2])-1
          if (_c <= 1):
            c_s[0] = 1
          if (_c <= 3):
            c_s[1] = 1
          cur_ret = np.append(cur_ret, c_s)
    
          pos = np.zeros(21)
          _p = int(cur[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          cur_ret = np.append(cur_ret, pos)
    
          pit = np.zeros(1)
          if (cur[k+4] == 1):
            pit[0] = 1
          else :
            pit[0] = 0
          cur_ret = np.append(cur_ret, pit)
    
          stat = stat_embed(int(cur[k+5]))
          cur_ret = np.append(cur_ret, stat)
    
          laptime = lt_embed(float(cur[k+6]))
          cur_ret = np.append(cur_ret, laptime)
    
          rando = np.zeros(1)
          cur_ret = np.append(cur_ret, rando)
  
        if (i == 0):
          cur_race_in = cur_ret
        else:
          cur_race_in = np.vstack([cur_race_in, cur_ret])
      np.save(db_dir + f'races_npy/{year}/{counter}_in.npy', cur_race_in)
  
    #if not os.path.exists(db_dir + f'races_npy/{year}/{round}_exp.npy'):
    if (True):
      for i in range(len(cur_race)-1):
        next = cur_race.iloc[i+1].values[1:142] # removes index and trim size
        # replace NaN with 0
        next[next != next] = 0
        for k in range(1, 141, 7):
          pos = np.zeros(21)
          _p = int(next[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          if (k == 1):
            next_ret = pos
          else:
            next_ret = np.append(next_ret, pos)
    
          pit = np.zeros(1)
          pit[0] = float(next[k+4]) / (len(cur_race)-1)
          next_ret = np.append(next_ret, pit)
    
          stat = stat_embed(int(next[k+5]))
          next_ret = np.append(next_ret, stat)
    
          laptime = lt_embed(float(next[k+6]))
          next_ret = np.append(next_ret, laptime)

        if (i == 0):
          exp = next_ret
        else:
          exp = np.vstack([exp, next_ret])
      np.save(db_dir + f'races_npy/{year}/{counter}_exp.npy', exp)

    counter += 1
  print(year)
  

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [ ]:
t = np.load(db_dir + 'races_npy/2001/0_in.npy')
t

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
len(t[0])

4051

## Dataset



v1:
Each data sample is a very long vector (4051) with the following features (and sizes):
- circuit id (130)
- lap number / total number of laps
- for each driver:
- - driver id (130)
- - driver standing (3)
- - constructor standing (2)
- - position (21)
- - pitting this lap OR laps/total number of laps till pitting (1) (input /output) (v1.1)
- - status (6)
- - laptime (32)
- - random (1)



In [ ]:
# to make feature size consistent, we use laptimes of top 20 drivers
class RaceDataset(torch.utils.data.Dataset):
  def __init__(self, dir):

    self.dir = dir
    self.year = 2001
    self.round = 1
    self.cache = {}

    # number of races this year
    self.cur_year = int(len(os.listdir(self.dir + f'{self.year}/'))/2)


    if (self.round < self.cur_year):
      self.cur_race = np.load(self.dir + f'{self.year}/{self.round-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.round-1}_exp.npy')
    else:
      self.cur_race = np.load(self.dir + f'{self.year}/{self.cur_year-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.cur_year - 1}_exp.npy')
    self.cache[f'{self.year}_{self.round}'] = self.cur_race


  def set_year(self, year):
    if (year == 2021):
      self.year = 2021
      return
    self.year = year
    self.cur_year = int(len(os.listdir(self.dir + f'{self.year}/'))/2)

  def set_round(self, round):
    self.round = round
    if (self.year == 2021):
      return # no data yet

    if (f'{self.year}_{self.round}' in self.cache):
      self.cur_race = self.cache[f'{self.year}_{self.round}']
      return
    # set round, if round > number of rounds in that season, set to the last round
    if (self.round < self.cur_year):
      self.cur_race = np.load(self.dir + f'{self.year}/{self.round-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.round-1}_exp.npy')
    else:
      self.cur_race = np.load(self.dir + f'{self.year}/{self.cur_year-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.cur_year - 1}_exp.npy')
    self.cache[f'{self.year}_{self.round}'] = self.cur_race

  def next_round(self):
    self.round += 1
    if (self.round < self.cur_year):
      self.set_round(self.round)
    else:
      self.round = 1
      self.set_year(self.year+1)
      self.set_round(self.round)


  def __len__(self):
    return len(self.cur_race[0])

  def __getitem__(self, i):
    cur_ret, next_ret = self.cur_race[0][i], self.cur_race[1][i]

    return (torch.from_numpy(cur_ret), torch.from_numpy(next_ret))

In [ ]:
ds = RaceDataset(db_dir + 'races_npy/')


In [ ]:
ds[0]

(tensor([0., 1., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 tensor([1., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64))

In [ ]:
len(ds[0][0])

4051

In [ ]:
len(ds[0][1])

1200

In [ ]:
ds[0][1][29*11+28]

tensor(0., dtype=torch.float64)

### Helper function for displaying dataset input/output

In [ ]:
def pos_df(lap_in, out, num_of_laps=1):
  df = pd.DataFrame(columns=['code', 'driver', 'position', 'laps till pitting', 'status', 'laptime'])
  _lap = lap_in.detach().clone().numpy()
  _o = out.detach().clone().numpy()
  _name, _loc, _country = circuit_info(np.argmax(_lap[:130]))
  for i in range(20):
    _d_idx = driver_unbed_idx(driver_unbed(_lap[131 + i * 196 : 131 + i * 196 + 130]))
    _num, _code, _fn, _ln, _, _ = driver_info(_d_idx)
    _pos = np.argmax(_o[i*60 : i*60 + 21]) + 1
    _pitting = _o[i*60+21] * num_of_laps
    if (_pitting == 0):
      _pitting = 'NA'
    _retired = False
    if (_pos == 21):
      _retired = True
    _status = stat_unbed(_o[i*60 + 22: i*60 + 28], _retired)
    _time = lt_unbed(_o[i*60 + 28:])
    df = df.append({
        'code': f'{_code}',
        'driver': f'{_fn} {_ln}',
        'position': _pos,
        'laps till pitting': _pitting,
        'status': _status,
        'laptime': _time
    }, ignore_index=True)

  df = df.sort_values(by=['position', 'laptime'], ascending=[True, False])
  return _name, _loc, _country, df


In [ ]:
ds.set_year(2001)
ds.set_round(1)

In [ ]:
_, _, _, df = pos_df(ds[1][0], ds[1][1])

In [ ]:
df

,code,driver,position,laps till pitting,status,laptime
0,MSC,Michael Schumacher,1,NA,No Problem,93.0
2,\N,Mika Häkkinen,2,NA,No Problem,93.0
3,\N,Heinz-Harald Frentzen,3,NA,No Problem,94.1
1,BAR,Rubens Barrichello,4,NA,No Problem,92.8
5,COU,David Coulthard,5,NA,No Problem,93.3
6,TRU,Jarno Trulli,6,NA,No Problem,95.0
4,SCH,Ralf Schumacher,7,NA,No Problem,98.0
7,VIL,Jacques Villeneuve,8,NA,No Problem,96.3
9,HEI,Nick Heidfeld,9,NA,No Problem,96.1
8,\N,Olivier Panis,10,NA,No Problem,95.9


In [ ]:
ds[0][0].shape

torch.Size([4051])

In [ ]:
del round

In [ ]:
# Returns a tensor with the size of in but content of out
def out_to_in(in_, out_, random=False, num_of_laps=50):
  _ret = in_.detach().clone().numpy()
  _o = out_.detach().clone().numpy()
  _o = _o.reshape([1200])
  _ret = _ret.reshape([4051])
  j = 0
  for i in range(0, 20):
    _ret[131 + i*196 + 135: i*196 + 131 + 195] = _o[j*60: (j+1) * 60]
    if (round(_o[j*60 + 21] * num_of_laps) <= 1):
      _ret[131 + i*196 + 130 + 26] = 1
    else:
      _ret[131 + i*196 + 130 + 26] = 0
    if (random):
      _ret[i * 196 + 131 + 195] = np.random.uniform(0, 20, [1])
    j += 1

  return torch.from_numpy(_ret).float()


In [ ]:
outfromin = out_to_in(ds[0][0], ds[0][1])

In [ ]:
print(outfromin)

In [ ]:
_, _, _, df2 = pos_df(outfromin, ds[0][1])


In [ ]:
df2

,code,driver,position,laps till pitting,status,laptime
0,MSC,Michael Schumacher,1,NA,No Problem,97.1
2,\N,Mika Häkkinen,2,NA,No Problem,98.4
3,\N,Heinz-Harald Frentzen,3,NA,No Problem,99.6
4,SCH,Ralf Schumacher,4,NA,No Problem,101.3
1,BAR,Rubens Barrichello,5,NA,No Problem,101.4
6,TRU,Jarno Trulli,6,NA,No Problem,102.3
5,COU,David Coulthard,7,NA,No Problem,102.8
7,VIL,Jacques Villeneuve,8,NA,No Problem,103.3
9,HEI,Nick Heidfeld,9,NA,No Problem,104.2
8,\N,Olivier Panis,10,NA,No Problem,104.6


## Model

In [ ]:
class RacePredictionModel(nn.Module):
    def __init__(self, input_size, output_size, lstm_hids, lstm_layers, dropout):
        super(RacePredictionModel, self).__init__()

        self.input_size = input_size
        self.lstm_layers = lstm_layers
        self.lstm_hids = lstm_hids

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hids, num_layers=lstm_layers, dropout=dropout, batch_first=True)

        self.fc = nn.Linear(lstm_hids, output_size)

        nn.init.xavier_uniform_(self.fc.weight.data)
        for name, params in self.lstm.named_parameters():
            if name[:6] == 'weight':
                nn.init.xavier_uniform_(params)
            elif name[:4] == 'bias':
                nn.init.constant_(params, 0.0)

    def zero_states(self, batchsize=1):
        hidden_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hids)
        cell_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hids)
        return (hidden_state, cell_state)

    def forward(self, ins, prev_states=None):
        lstm_outs, next_states = self.lstm(ins, prev_states)
        outs = self.fc(lstm_outs)
        return outs, next_states

### Helper functions for training #1

In [ ]:

# Trains model on data from 2001 to 2019, 2020 is reserved for testing
# Method 1
# Training procedure:
#   for each race:
#     set zero states,
#     for each lap:
#       feed input from dataset,
#       calculate loss from output,
#       back propagate,
def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0

    ds.set_year(2001)
    ds.set_round(1)

    while (ds.year != 2021):
      hid_state, cell_state = model.zero_states()
      states = hid_state.to(device), cell_state.to(device)
      for i in range(len(ds)):
          opt.zero_grad()
          lap_in, lap_exp = ds[i]
          #if (i == 0):
          #  lap_in, lap_exp = ds[i]
          #else:
          #  _, lap_exp = ds[i]
          #  lap_in = out_to_in(lap_in.cpu(), lap_out.cpu()).to(device)

          # fix shape and type
          # input size should be (batch, seq_len, input_size)
          # we are not using batches
          #if (i == 0):
          if (True):
            lap_in = lap_in.unsqueeze(0)
            lap_exp = lap_exp.unsqueeze(0)
          lap_in = lap_in.unsqueeze(0).float()
          lap_exp = lap_exp.unsqueeze(0).float()

          lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
          lap_out, states = model(lap_in, states)
          loss = crit(lap_out, lap_exp)
          for s in states:
              s.detach_()
          loss.backward()
          if (math.isnan(loss)):
            print('Loss is nan')
            print(f'Year: {ds.year}')
            print(f'round: {ds.round}')
            print(f'lap: {i}')
            print(lap_in)
            input()
          opt.step()
          total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
          total_count += 1 #lap_in.size(0)
      ds.next_round()

    sched.step()
    return total_loss/total_count

def run_test(model, ds, crit):
    model.eval()
    total_loss, total_count = 0, 0

    ds.set_year(2020)
    ds.set_round(1)

    hid_state, cell_state = model.zero_states()
    states = hid_state.to(device), cell_state.to(device)

    for i in range(len(ds)):
      #lap_in, lap_exp = ds[i]
      if (i == 0):
        lap_in, lap_exp = ds[i]
      else:
        _, lap_exp = ds[i]
        lap_in = out_to_in(lap_in.cpu(), lap_out.cpu()).to(device)
      #if (i == 0):
      if (True):
        lap_in = lap_in.unsqueeze(0)
        lap_exp = lap_exp.unsqueeze(0)
      lap_in = lap_in.unsqueeze(0).float()
      lap_exp = lap_exp.unsqueeze(0).float()
      lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
      lap_out, states = model(lap_in, states)
      loss = crit(lap_out, lap_exp)
      total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
      total_count += 1 #lap_in.size(0)

    return total_loss/total_count

def run_all(model, ds, crit, opt, sched, versionId, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs', unit='ep'):
      train_loss = run_train(model, ds, crit, opt, sched)
      test_loss = run_test(model, ds, crit)
      tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}  test loss {test_loss:.6f}')
      if epoch % 5 == 0:
        torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{epoch}-loss-{test_loss:.2f}.pth')


### Training

In [ ]:
# only init once, cause "caching"
ds = RaceDataset(db_dir + 'races_npy/')


In [ ]:
device = torch.device('cuda:0')
#device = torch.device('cpu')
model = RacePredictionModel(4051, 1200, 1200, 2, 0.2)
model.to(device)
crit = nn.MSELoss().to(device)
opt = optim.Adam(model.parameters(), lr=0.001)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.2)
#dl = CustomeLoader(ds, 32)

In [ ]:
run_all(model, ds, crit, opt, sched, 47.1, 10)
#run_all_2(model, ds, dl, 32, crit, opt, sched, 32, 10)

vId | training method | ds | ds cls| input size | hid size | layers | dropout | crit | opt | sched | epochs | train loss | test loss
--- | --- | --- | --- | --- | --- | --- |--- | --- | --- | --- | ---| --- | ---
43 | 1 | v1 | v1| 4051 | 1200 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0477|0.077  
44 | 1 | v1 | v1| 4051 | 1600 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0515|0.0695  
44.1 | 1 | v1 | v1| 4051 | 1600 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 5|0.0518|
44.2 | 1 | v1 | v1| 4051 | 1200 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 5|0.0480|
45 | 1 | v1 | v1| 4051 | 2000 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0537|0.0665  
46 | 1 | v1 | v1| 4051 | 800 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0480|0.0661  
47 | 1 | v1, from 2001 | v1| 4051 | 1600 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0500|0.0739  
47.1 | 1 | v1, from 2001 | v1| 4051 | 1200 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 6|0.0513|0.0  


43: moderate movement  
44: more retirement, less overfitted and lower loss but in practice similar   
45: more than half the grid retires, less overfitted and lower loss but in practice similar   
46:   
decimal places: trained data including 2020

In [ ]:
run_test(model, ds, crit)

0.06231908148653071

In [ ]:
model.eval()
ds.set_year(2020)
ds.set_round(20)
p, n = ds[10]
p = p.to(device)
hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)
out = out.squeeze().squeeze()
print(out)
pos_df(p.to(cpu), out.to(cpu))


In [ ]:

cpu = torch.device('cpu')
model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/47.1-5-loss-0.07.pth'))
model.to(cpu)

RacePredictionModel(
  (lstm): LSTM(4051, 1200, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=1200, out_features=1200, bias=True)
)

In [ ]:
torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{epoch}-loss-{test_loss:.2f}.pth')

In [ ]:
cpu = torch.device('cpu')
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/34-5-loss-0.06.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/41-10-loss-0.06.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/43-11-loss-0.08.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/44-11-loss-0.07.pth'))
model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/47.1-5-loss-0.07.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/44.2-5-loss-0.07.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/11-5-loss-37.31.pth'))
model.eval()
ds.set_year(2020)
ds.set_round(30)
p, n = ds[0]
p = p.to(device)
hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)
out = out.squeeze().squeeze()
for i in range(50):
  out, s = model(out_to_in(p.to(cpu), out.to(cpu), True).to(device).unsqueeze(0).unsqueeze(0).float(), s)
  out = out.squeeze().squeeze()
  a, b, c, d = pos_df(p.to(cpu), out.to(cpu), len(ds))
  print(a,b,c)
  display(d)
out = out.detach().to(cpu)
d

Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,14.154102,No Problem,102.6
1,BOT,Valtteri Bottas,2,15.111991,No Problem,109.9
2,HAM,Lewis Hamilton,3,14.915105,No Problem,100.0
3,NOR,Lando Norris,4,14.131464,No Problem,109.0
4,ALB,Alexander Albon,5,13.763487,No Problem,109.0
5,SAI,Carlos Sainz,6,12.569674,No Problem,100.0
6,KVY,Daniil Kvyat,7,13.272663,No Problem,109.1
7,STR,Lance Stroll,7,13.730545,No Problem,101.3
8,GAS,Pierre Gasly,9,12.129380,No Problem,109.6
9,OCO,Esteban Ocon,10,14.577199,No Problem,100.0


Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,13.184612,No Problem,102.6
1,BOT,Valtteri Bottas,2,13.888947,No Problem,108.6
2,HAM,Lewis Hamilton,3,13.444796,No Problem,100.0
3,NOR,Lando Norris,4,12.915915,No Problem,109.0
4,ALB,Alexander Albon,5,12.415240,No Problem,109.0
5,SAI,Carlos Sainz,6,11.480988,No Problem,100.0
6,KVY,Daniil Kvyat,7,11.970304,No Problem,108.4
7,STR,Lance Stroll,7,12.283481,No Problem,101.3
8,GAS,Pierre Gasly,9,10.863786,No Problem,109.0
12,VET,Sebastian Vettel,9,12.436596,No Problem,100.0


Yas Marina Circuit Abu Dhabi UAE


...

Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,7.199877,No Problem,98.3
2,HAM,Lewis Hamilton,1,3.851373,No Problem,98.0
1,BOT,Valtteri Bottas,2,6.126218,No Problem,98.7
3,NOR,Lando Norris,4,5.140823,No Problem,98.9
4,ALB,Alexander Albon,5,4.764532,No Problem,98.0
7,STR,Lance Stroll,7,4.603866,No Problem,109.8
6,KVY,Daniil Kvyat,7,5.665322,No Problem,108.4
10,RIC,Daniel Ricciardo,8,5.621929,No Problem,100.2
9,OCO,Esteban Ocon,10,5.936216,No Problem,109.2
13,GIO,Antonio Giovinazzi,14,3.485741,No Problem,100.0


Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,7.403899,No Problem,98.3
2,HAM,Lewis Hamilton,1,3.799132,No Problem,98.0
1,BOT,Valtteri Bottas,2,5.845746,No Problem,98.7
3,NOR,Lando Norris,4,4.838649,No Problem,98.9
4,ALB,Alexander Albon,5,4.347290,No Problem,98.0
6,KVY,Daniil Kvyat,7,5.266117,No Problem,198.4
7,STR,Lance Stroll,7,4.503398,No Problem,109.6
10,RIC,Daniel Ricciardo,8,5.849525,No Problem,100.2
9,OCO,Esteban Ocon,10,5.793567,No Problem,109.2
13,GIO,Antonio Giovinazzi,15,3.158238,No Problem,100.0


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,7.403899,No Problem,98.3
2,HAM,Lewis Hamilton,1,3.799132,No Problem,98.0
1,BOT,Valtteri Bottas,2,5.845746,No Problem,98.7
3,NOR,Lando Norris,4,4.838649,No Problem,98.9
4,ALB,Alexander Albon,5,4.347290,No Problem,98.0
6,KVY,Daniil Kvyat,7,5.266117,No Problem,198.4
7,STR,Lance Stroll,7,4.503398,No Problem,109.6
10,RIC,Daniel Ricciardo,8,5.849525,No Problem,100.2
9,OCO,Esteban Ocon,10,5.793567,No Problem,109.2
13,GIO,Antonio Giovinazzi,15,3.158238,No Problem,100.0


In [ ]:
pos_df(p.to(cpu), n.to(cpu))

('Yas Marina Circuit',
 'Abu Dhabi',
 'UAE',
    code              driver position  pitting      status  laptime
 0   VER      Max Verstappen        1      0.0  No Problem    103.0
 1   BOT     Valtteri Bottas        2      0.0  No Problem    105.0
 2   HAM      Lewis Hamilton        3      0.0  No Problem    106.0
 3   NOR        Lando Norris        4      0.0  No Problem    107.0
 4   ALB     Alexander Albon        5      0.0  No Problem    108.0
 5   SAI        Carlos Sainz        6      0.0  No Problem    109.0
 6   KVY        Daniil Kvyat        7      0.0  No Problem    110.0
 7   STR        Lance Stroll        8      0.0  No Problem    111.0
 9   OCO        Esteban Ocon        9      0.0  No Problem    112.0
 8   GAS        Pierre Gasly       10      0.0  No Problem    112.0
 10  RIC    Daniel Ricciardo       11      0.0  No Problem    113.0
 12  VET    Sebastian Vettel       12      0.0  No Problem    113.0
 11  LEC     Charles Leclerc       13      0.0  No Problem    114.0
 14

In [ ]:
c = torch.cat((ds[0][0].unsqueeze(0).unsqueeze(0), ds[1][0].unsqueeze(0).unsqueeze(0)), 0)

In [ ]:
c.shape

torch.Size([2, 1, 5330])

In [ ]:
o, s = model(c.to(device).float(), states)

In [ ]:
o.shape

torch.Size([1, 2, 1600])

In [ ]:
s[0].shape

torch.Size([10, 1, 128])

In [ ]:
s[1].shape

torch.Size([10, 1, 128])

In [ ]:
oo, ss = model(ds[0][0].unsqueeze(0).unsqueeze(0).float().to(device), states)

In [ ]:
oo.shape

torch.Size([1, 1, 1600])

In [ ]:
ss[0].shape

torch.Size([10, 1, 128])

In [ ]:
len(ss)

2